In [4]:
# We step up the game here. 
# Develop and evaluate deep learning models for object recognition in Keras
# This is a difficult task because of the infinite number of permutations of objects, positions and lighting
# We use CIFAR (Canadian institute for Advanced Research)

In [12]:
# Load data
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [13]:
seed = 7
numpy.random.seed(seed)

In [14]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [15]:
# The pixel values are in the range 0-255 for each red, geen an dblue channels. 
# As it is good practice to work with normalize data - as the input value are well understood.
# we there normalize inputs from 0-255 to 0.0 - 1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [16]:
# The output variables are defined as a vector of integers from 0 - 1 for each class. We can use a one hot encoding to transfor 
# them int a binary matrix in order to best model the classification problem.
# We know there are 10 classes for this problem - we cna there expect the binary matrix to have a width of 10
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [17]:
# Let’s start off by defining a simple CNN structure as a baseline and evaluate how well it
# performs on the problem. We will use a structure with two convolutional layers followed by
# max pooling and a flattening out of the network to fully connected layers to make predictions.
# Our baseline network structure can be summarized as follows:
# 1. Convolutional input layer, 32 feature maps with a size of 3 × 3, a rectifier activation
# function and a weight constraint of max norm set to 3.
# 2. Dropout set to 20%.
# 3. Convolutional layer, 32 feature maps with a size of 3 × 3, a rectifier activation function
# and a weight constraint of max norm set to 3.
# 4. Max Pool layer with the size 2 × 2.
# 5. Flatten layer.
# 6. Fully connected layer with 512 units and a rectifier activation function.21.3. Simple CNN for CIFAR-10
# 7. Dropout set to 50%.
# 8. Fully connected output layer with 10 units and a softmax activation function.
# A logarithmic loss function is used with the stochastic gradient descent optimization algorithm
# configured with a large momentum and weight decay, starting with a learning rate of 0.01. A
# visualization of the network structure is provided below.

In [18]:
# create the model
model = Sequential()
# Output layer (10 outputs)
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# Dropout layer (20%)
model.add(Dropout(0.2))
# Hidden layer (512 Neurons)
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
# Max pooling layer (2 X 2)
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
# Convolutional Layer (32 maps, 3 x 3)
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
# Drop out
model.add(Dropout(0.5))
# visible layer
model.add(Dense(num_classes, activation='softmax'))

In [19]:
# compile
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [20]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

In [21]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25


   32/50000 [..............................] - ETA: 80519s - loss: 2.2854 - acc: 0.2188

  160/50000 [..............................] - ETA: 16079s - loss: 2.3260 - acc: 0.1437

  288/50000 [..............................] - ETA: 8920s - loss: 2.3268 - acc: 0.1458 

  448/50000 [..............................] - ETA: 5722s - loss: 2.3215 - acc: 0.1295

...

49472/50000 [============================>.] - ETA: 0s - loss: 0.2377 - acc: 0.9166

49632/50000 [============================>.] - ETA: 0s - loss: 0.2375 - acc: 0.9167

49792/50000 [============================>.] - ETA: 0s - loss: 0.2375 - acc: 0.9168

49952/50000 [============================>.] - ETA: 0s - loss: 0.2373 - acc: 0.9169

50000/50000 [==============================] - 18s - loss: 0.2374 - acc: 0.9169 - val_loss: 1.0866 - val_acc: 0.7010

In [22]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)

In [23]:
print('Accuracy: %.2f%%' % (scores[1]*100))

Accuracy: 70.10%


[[ 22.  28.]
 [ 49.  64.]]
